In [1]:
import json
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import copy
import os
from PIL import Image
import timm
from tqdm import tqdm
import torch.nn.functional as F

In [2]:
with open('result.json', 'r') as f:
    pred = json.load(f)
    
with open('/opt/ml/detection/dataset/test.json', 'r') as f:
    images = json.load(f)

In [3]:
transform_test = transforms.Compose([
    transforms.Resize(size=(224, 244)),
    transforms.ToTensor(),
    transforms.Normalize(mean=torch.tensor([0.4850, 0.4560, 0.4060]), std=torch.tensor([0.2290, 0.2240, 0.2250]))]
)

In [4]:
class EvalDataset(Dataset):
    
    def __init__(self, images_json, pred, transform):
        
        images = {}
        for img in images_json['images']:
            images[img['id']] = img['file_name']
        
        root = '/opt/ml/detection/dataset/'
        self.pred = copy.deepcopy(pred)
        for anno in self.pred:
            anno['img_path'] = os.path.join(root, images[anno['image_id']])
        self.transform = transform
    
    def __len__(self):
        return len(self.pred)
    
    def __getitem__(self, idx):
        
        anno = self.pred[idx]
        img_path = anno['img_path']
        x1, y1, w, h = anno['bbox']
        img = Image.open(img_path).convert('RGB')
        img = img.crop((x1-w*0.05, y1-h*0.05, x1 + 1.05* w ,y1 + 1.05*h))
        img = self.transform(img)
        
        return img, anno['category_id']

In [5]:
ds = EvalDataset(images, pred, transform_test)
dl = DataLoader(ds, batch_size=64, num_workers=4, drop_last=False, shuffle=False)

tensor([7, 7, 7, 7, 7, 7, 1, 7, 7, 5, 5, 5, 2, 5, 5, 1, 6, 5, 5, 7, 1, 5, 5, 7,
        1, 5, 7, 5, 5, 5, 1, 7, 5, 7, 5, 5, 1, 1, 1, 7, 0, 5, 5, 7, 7, 0, 5, 5,
        5, 6, 5, 1, 5, 5, 0, 2, 7, 1, 0, 7, 6, 1, 7, 5])


In [6]:
from spam.utils import imshow
a = iter(dl)
batch = next(a)
imshow(batch[0])
print(batch[1])

In [7]:
device = 'cuda'

In [9]:
import pickle

In [10]:
model = timm.create_model('tf_efficientnet_b0', pretrained=False, num_classes=11)
model.to(device)

model.load_state_dict(torch.load(f'./save/split1.pt'))
model.eval()
print('ready')

res = []

with torch.no_grad():
    for b, i in tqdm(dl):

        b = b.cuda()

        output = model(b)
        output = F.softmax(output, dim=-1)
        output = output.cpu().numpy()
        res.append(output)

res = np.concatenate(res, axis=0)

with open('results/1.pkl', 'wb') as f:
    pickle.dump(res, f)

ready


100%|██████████| 8687/8687 [1:15:07<00:00,  1.93it/s]
